In [ ]:
# %reload_ext autoreload
# %autoreload 2
#!pip install torchinfo
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

from src import autoencoder as ae
from torchinfo import summary

from glob import glob, escape
import torch
from torch.utils.data import DataLoader
from src.datagen_aug import datagen, LipGanDS
from pathlib import Path

from tqdm.auto import tqdm
import numpy as np

datas = {'data_root/val':1}
USE_TANH = True
device = 'cuda'

from moviepy import editor as mpe
import IPython.display as ipd


class args:
    mel_ps = 80 
    img_size = 288
    batch_size = 4 
    mel_step_size = 108
    mask_ver =(9) 
    mask_img_trsf_ver = 0
    num_ips = 2
    # lr = 5e-3
    num_workers = 4
    mel_trsf_ver = -1
    
def load_model(weight_path):
    model = ae.Speech2Face(3, (3, args.img_size, args.img_size), (1, 96, args.mel_step_size))
    model.load_state_dict(torch.load(weight_path, map_location=device))
    return model.to(device).eval()

        
def get_dl():
        
    args.val_images = []
    for data_root_val, stride in datas.items():
        args.val_images += sorted(glob(f'{data_root_val}/*/*.jpg'))[:-60:stride]
    for data_root_val, stride in datas.items():
        args.val_images += sorted(glob(f'{data_root_val}/*/*.jpg'))[60::stride]
    print('len(val_images):', len(args.val_images))
    ds = LipGanDS(args, phase='val')
    ds[0]
    batch_size = args.batch_size
    dl = DataLoader(dataset=ds, batch_size=batch_size, num_workers=args.num_workers, shuffle=False, drop_last=True)
    return dl
dl = get_dl()
#for _ in dl:
#    pass
len(dl)


In [ ]:
#ds=dl.dataset
#img_gt, mel, ips = next(iter(dl))

In [ ]:
#from PIL import Image
##img_gt_n = to_img(img_gt)
##display(Image.fromarray(img_gt_n[0][:,:,::-1]))
##t = to_img(ips[0][:,:,0:3])
##display(Image.fromarray(t[:,:,::-1]))
#
#
#frames = []        
#for img_gt, mel, ips  in tqdm(dl):
#    
#    audio = mel.unsqueeze(1).to(device)
#    ips = ips.to(device).permute(0,3,1,2)
#    ips = ips.permute(0,2,3,1)
#    b,h,w,c = ips.shape
#    print('ips.shape', ips.shape)
#    ipss = [e.reshape(h,w,3,c//3).permute(2,0,1,3) for e in ips]
#    print('ipss[0].shape', ipss[0].shape)
#    ips2 = torch.stack(ipss, axis=1)
#    print('ips2.shape', ips.shape)
#    img_gt, ips3 = to_img(img_gt), [to_img(e) for e in ips2]
#    print('len(ips3):', len(ips3), 'ips3[0].shape', ips3[0].shape)
#    
#    frames.append(np.concatenate(list(reversed(ips3 + [img_gt])), axis=2))
#    break
#print(frames[0].shape)
##frames = np.concatenate(frames, axis=0)[:,:,:,[2,1,0]]
##frames[0].shape
##Image.fromarray(frames[0])

In [ ]:
#frames2 = np.concatenate(frames, axis=0)[:,:,:,[2,1,0]]
#print(frames2.shape)
#Image.fromarray(frames2[0])
#for i in range(3):
#    t = to_img(ips[0][:,:,i*3:i*3+3])
#    print(t.shape)
#    display(Image.fromarray(t[:,:,::-1]))
#for im in ipss[0]:
#    t = to_img(im)
#    print(t.shape)
#    display(Image.fromarray(t[:,:,::-1]))
#    

In [ ]:
def to_img(t):
    img = t.cpu().numpy().astype(np.float64)
    img = ((img / 2.0) + 0.5) * 255.0
    img = np.clip(img, 0.0, 255.0).astype(np.uint8)
    return img

def get_generated_frame(model, dl, device):
    frames = []
    model.eval()
        
    for img_gt, mel, ips  in tqdm(dl):
        
        audio = mel.unsqueeze(1).to(device)
        ips = ips.to(device).permute(0,3,1,2)
       
        with torch.no_grad():
            img_pred = model(ips, audio)
            
        gen_face  = to_img(img_pred.permute(0,2,3,1))
        ips = ips.permute(0,2,3,1)
        b,h,w,c = ips.shape
        #print('ips.shape', ips.shape)
        ipss = [e.reshape(h,w,3,c//3).permute(2,0,1,3) for e in ips]
        #print('ipss[0].shape', ipss[0].shape)
        ips = torch.stack(ipss, axis=1)
        #print('ips.shape', ips.shape)
        img_gt, ips = to_img(img_gt), [to_img(e) for e in ips][::-1]
        
        frames.append(np.concatenate(list(reversed(ips + [gen_face, img_gt])), axis=2))
    frames = np.concatenate(frames, axis=0)[:,:,:,[2,1,0]]
    return frames

def view_video(frames, vpath):
    if frames is None and Path(vpath).exists():
        display(ipd.Video(vpath))
        return
        
    vc = mpe.ImageSequenceClip(list(frames), fps=30)
    ffmpeg_params = None
    #ffmpeg_params=['-acodec', 'aac', '-preset', 'veryslow', '-crf', '17']
    Path(vpath).parent.mkdir(parents=True, exist_ok=True)
    vc.write_videofile(vpath, ffmpeg_params=ffmpeg_params, logger=None)
    display(ipd.Video(vpath))

def view_val(weight_path, vpath):
    print(weight_path)
    #dl = get_dl()
    if not Path(vpath).exists():
        model = load_model(weight_path)
        frames = get_generated_frame(model, dl, device)
    else:
        frames = None
    view_video(frames, vpath)

def infer_model(PRJNAME, start=-1, end=-2, stride=-1):
    weight_paths = sorted(glob(escape(f'weights/{PRJNAME}') + '/*.pth'))
    for i, weight_path in enumerate(weight_paths[start:end:stride]):
        view_val(weight_path, f'gen_videos/{PRJNAME}_{Path(weight_path).name}.mp4')

In [ ]:
from IPython.display import clear_output
import time

PRJNAME = 'Adam_Grad_9_bs-144_lr-1e-05_mel_ps_80_2021-07-10 23:59'

#while True:
if True:
    infer_model(PRJNAME, start=0,  end=100, stride=1)
#    time.sleep(30*60)
#    clear_output()

In [ ]:
PRJNAME = 'Adam_Base_9_bs-144_lr-1e-05_mel_ps_80_2021-07-11 02:22'
while True:
    infer_model(PRJNAME, start=0,  end=100, stride=1)
    time.sleep(30*60)
    clear_output()